# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



In [1]:
// initializevar path = require('path');var fs = require('fs');var getCellsOrDirectory = (filename) => {    const fname = filename.replace('.ipynb', '');    if (fs.existsSync(fname)        && fs.statSync(fname).isDirectory()) {        const files = fs.readdirSync(fname)            .sort((a, b) => parseInt(a.replace(/[^0-9]/ig, '')) - parseInt(b.replace(/[^0-9]/ig, ''))) //sort numerically            .filter(f => f.indexOf('.js') > -1);        var code = files.map(f => fs.readFileSync(path.join(fname, f)).toString());        return Promise.resolve(code);    }    else {        return getCells(filename, ['javascript', 'code'])            .then(cells => cells.map(c => c.source.join('')));    }};// provide a function for importing any notebook as a module and executing itvar executeNotebook = (notebook, ctx = {}) => {    var filename = path.resolve(notebook);    var dirname = path.resolve(path.dirname(filename));    var oldDir = process.cwd();    process.chdir(dirname);    console.log(filename);    // only read javascript kernels    return getCellsOrDirectory(filename)        .then(cells => {            // TODOCUMENT: Notebooks are imported in to the same context so filename and dirname are overwritten.            if (cells.length > 0) {                cells[0] = 'var __filename=' + JSON.stringify(filename) + ';'                    + 'var __dirname=' + JSON.stringify(dirname) + ';'                    + 'process.chdir(' + JSON.stringify(dirname) + ');\n'                    + cells[0];            }            return cells;        })        .then(cells => runAllPromises(cells            .map((c, i) => promiseNewContext(c, ctx, notebook + '[' + i + ']'))))        .then(r => {            const results = assignResults(r, notebook);            process.chdir(oldDir);            return results;        })        .catch(e => console.log(e))};// output nothing here, cached version of this function is assigned below.if (typeof imported !== 'object' || imported === null) {    var imported = {};}var fuseSearch, interpretObject;var importCells = (queries, ctx) => {    return Promise.resolve(        typeof imported['interpret all notebooks.ipynb'] !== 'undefined'            ? imported['interpret all notebooks.ipynb']            : executeNotebook('interpret all notebooks.ipynb'))        .then(r => {            fuseSearch = r['fuseSearch'];            interpretObject = r['interpretObject'];        })        .then(() => {            return typeof queries === 'string'                ? fuseSearch(queries)                : Promise.all(queries.map(fuseSearch))        })        .then(r => {            return typeof queries === 'string'                ? interpretObject([r[0]])                : interpretObject(r.map(r => r[0]))        })        .then(r => r.map(r => {            if (typeof r === 'undefined') {                return r;            }            return promiseNewContext(r.fresher, ctx);        }))        .then(r => typeof queries === 'string' ? r[0] : r)        .catch(e => console.log(e));};// How to test if a notebook has already been imported?var importNotebook = (notebook, ctx) => {    if (typeof notebook === 'undefined') {        return Promise.resolve({});    }    var filename = path.basename(notebook);    if (typeof imported[filename] !== 'undefined') {        return Promise.resolve(imported[filename]);    }    const queries = ((/\[(.*?)]/ig).exec(filename) || {})[1];    if (typeof queries !== 'undefined') {        return importCells(queries, ctx);    }    return executeNotebook(notebook, ctx);};// $$.done(); for our dumb parser

What do I need to have installed?

How to sanity check the environment?


In [2]:
// initialize// $$.done() for our dumb parser

How to stream json using a {match} function?

In [3]:
// initializevar fs = require('fs');var JSONStream = require('JSONStream');var streamJson = (file, parse, match = ((c) => true)) => {    var file = fs.createReadStream(file)        .pipe(JSONStream.parse(parse));    file.on('data', (m) => match(m));    // create a promise out of this stream    return new Promise((resolve, reject) => {        file.on('error', e => reject(e, file));        file.on('close', () => resolve(file))    });};// $$.done() for our dumb parser


How to parse cells of {types} from a notebook?



In [4]:
// initialize// How to use getCells?var getCells = (notebook, types = ['*', 'code']) => {    var cells = [];    var kernel;    return streamJson(notebook, [true, {emitPath: true}], (match) => {        if (match.path[0] === 'metadata'            && match.path[1] === 'kernelspec') {            kernel = match.value;        } else if (match.path[0] === 'cells'            && types.indexOf(match.value.cell_type) > -1) {            cells[cells.length] = match.value;        }    }).then(() => {        if (types.indexOf(kernel.language) === -1            && types.indexOf('*') === -1) {            console.log('tried to import ' + types + JSON.stringify(kernel));            return [];        }        return cells.map(c => Object.assign(c, {language: kernel.language}));    });};// $$.done() for our dumb parser

How to run all promises sequentially?



In [5]:
// initializevar runAllPromises = (promises) => {    return promises.reduce((promise, func) => {        return promise.then(result => {            return (typeof func == 'function'                ? (new Promise(func)) : Promise.resolve(func))                .then(Array.prototype.concat.bind(result));        });    }, Promise.resolve([]));};// $$.done() for our dumb parser

How to run all code cells in jupyter?

In [6]:
// initializeif (typeof imported !== 'object' || imported === null) {    var imported = {};}// cache results when notebooks are processedvar assignResults = (results, notebook) => {    var filename = path.basename(notebook);    results = results.reduce((obj, r, i) => {        if (typeof r === 'function') {            obj[filename + '[' + i + ']'] = r;            obj[r.name] = obj[filename + '[' + i + ']'];        }        else if (typeof r === 'object' && r !== null            && typeof r['error'] !== 'undefined') {            obj[filename + '[' + i + ']'] = r['error'];            console.log(filename + '[' + i + ']' + '\n' + r['error']['evalue']);        }        return obj;    }, {});    // TODO: offer results as an express service instead of global context?    imported[filename] = results;    return results;};// $$.done() for our dumb parser

How to import a {notebook}?

How to test the import feature?


In [7]:
// initialize// $$.done() for our dumb parser

How to load interpreter?

How to search notebooks?


In [8]:
// initialize// save for later usevar fuseSearch, interpretObject;var runInNewContext = (code, ctx) => void 0;var interpret = (queries, root = '') => {    return (typeof fuseSearch === 'undefined'        ? importNotebook('interpret all notebooks.ipynb')            .then((f) => {                fuseSearch = f['fuseSearch'];                interpretObject = f['interpretObject'];            })        : Promise.resolve())    // use Promise.all to do all of your code searches up front        .then(() => {            return typeof queries === 'string'                ? fuseSearch(queries)                : Promise.all(queries.map(fuseSearch))        })        .then(r => {            if (r.length === 0) {                throw 'Nothing found for ' + queries;            }            return typeof queries === 'string'                ? interpretObject([r[0]])                : interpretObject(r.map(r => r[0]))        })        .then(r => r.map(r => {            if (typeof r === 'undefined') {                return r;            }            // must have a unique id for each unique cell so that            // individual cells can serve as modules as well as notebooks            // adding the cell id as a part of the filename            r.runInNewContext = (ctx) => runInNewContext(                r.fresher,                Object.assign(ctx || {}, {useCache: false, __filename: r.filename + r.id}),                {filename: r.filename + r.id}, false);            return r;        }))        .then(r => typeof queries === 'string' ? r[0] : r)        .catch(e => console.log(e))};// $$.done() for our dumb parser

Import arbitrary code in to the current context?

In [9]:
// initializevar path = require('path');var requireCode = (code, pathToCode, global) => {    var Module = require('module').Module;    // must have a new name for every generation otherwise cache will be returned    var filepath = path.resolve(process.cwd(), pathToCode);    var dirname = path.dirname(filepath);    var cachedModule = Module._cache[filepath];    if (cachedModule && global.useCache !== false) {        global.module = cachedModule;        return cachedModule.exports;    }    var mod = new Module(filepath, global.module);    global.module = mod;    Module._cache[filepath] = mod;    mod.filename = filepath;    mod.paths = Module._nodeModulePaths(dirname);    mod._compile(code, filepath);    mod.loaded = true;    global.exports = mod.exports;    return mod.exports;};// $$.done() for our dumb parser

Import arbitrary code in to a new context?

In [10]:
// initializevar path = require('path');var vm = require('vm');var runner = function (resolve, reject, file) {    'use strict';    this.resolve = resolve;    this.reject = reject;    this.isAsync = false;    this.returnVal = null;    this.file = file;    this.async = () => {        this.isAsync = true;    };    this.sendResult = (r) => {        if (this.isAsync) {            return this.resolve(r);        } else {            this.returnVal = r;        }    };    this.sendError = (e) => {        console.log(e);        this.reject(e);    }};var promiseNewContext = (c, ctx, newFilename) => {    return (resolve, reject) => {        var isPromise = false;        var run = new runner(resolve, reject, newFilename);        var newCtx = Object.assign(ctx || {}, {            __filename: newFilename,            $$: run        });        const result = runInNewContext(c, newCtx, {filename: newFilename}, false);        if (typeof result === 'object' && result !== null) {            if (result.constructor === Promise.prototype.constructor) {                isPromise = true; // will return result of promise            }        }        return Promise.resolve(result)            .then(r => {                if (!run.isAsync) {                    return resolve(run.returnVal || r);                } else if (isPromise) {                    return resolve(run.returnVal || r);                }                return r;            })            .catch(e => reject(e))    }    /*    .then(r => {        if (typeof r === 'function') {            newCtx[filename + '[' + i + ']'] = r;            newCtx[r.name] = obj[filename + '[' + i + ']'];        }        return r;    });    */};var runInNewContext = (code, ctx, options, isModule = true) => {    var tmpGlobal = ctx || {};    tmpGlobal.global = tmpGlobal;    Object.assign(tmpGlobal, ctx);    Object.assign(tmpGlobal, {        requireCode: requireCode,        moduleCode: '"use strict";\n' + code,        projectPath: ctx.__filename,        'console': console,        'process': process,        'Promise': Promise,        'Buffer': Buffer,        'clearImmediate': clearImmediate,        'clearInterval': clearInterval,        'clearTimeout': clearTimeout,        'setImmediate': setImmediate,        'setInterval': setInterval,        'setTimeout': setTimeout,        '__filename': path.basename(ctx.__filename),        '__dirname': path.dirname(ctx.__filename),        'require': require    });    Object.assign(options || {}, {lineOffset: -1});    if (!isModule) {        tmpGlobal.module = {};    }    //var result = vm.runInNewContext('"use strict";(requireCode(moduleCode, projectPath, global));', tmpGlobal);    var result = vm.runInNewContext(isModule        ? 'requireCode(moduleCode, projectPath, global);'        : tmpGlobal.moduleCode, tmpGlobal, options);    return tmpGlobal.global.module.exports || result;};// $$.done() for our dumb parser

In [11]:
// initializeif (typeof imported !== 'object' || imported === null) {    var imported = {};}imported['import notebook.ipynb'] = {    'import notebook.ipynb[2]': streamJson,    'streamJson': streamJson,    'import notebook.ipynb[3]': getCells,    'getCells': getCells,    'import notebook.ipynb[4]': runAllPromises,    'runAllPromises': runAllPromises,    'import notebook.ipynb[7]': importNotebook,    'import': importNotebook,    'import notebook.ipynb[8]': imported,    'imported': imported,    'import notebook.ipynb[9]': interpret,    'interpret': interpret,    'import notebook.ipynb[11]': runInNewContext,    'runInNewContext': runInNewContext};module.exports = imported['import notebook.ipynb'];// $$.done() for our dumb parser

{ 'import notebook.ipynb[2]': [Function: streamJson],
  streamJson: [Function: streamJson],
  'import notebook.ipynb[3]': [Function: getCells],
  getCells: [Function: getCells],
  'import notebook.ipynb[4]': [Function: runAllPromises],
  runAllPromises: [Function: runAllPromises],
  'import notebook.ipynb[7]': [Function: importNotebook],
  import: [Function: importNotebook],
  'import notebook.ipynb[8]': { 'import notebook.ipynb': [Circular] },
  imported: { 'import notebook.ipynb': [Circular] },
  'import notebook.ipynb[9]': [Function: interpret],
  interpret: [Function: interpret],
  'import notebook.ipynb[11]': [Function: runInNewContext],
  runInNewContext: [Function: runInNewContext] }

These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

